# Proyecto Final

## Programación

## Profesor: Ing. Andrés Mena A.

### **Nombre del estudiante: Stephanny Garita Vega**

**Objetivo principal:

Analizar el ciclo de vida de los prospectos generados en la campaña de ventas del IIIQ 2025, midiendo el tiempo que transcurre desde su ingreso hasta el primer contacto, seguimientos y la conversión final, con el fin de identificar patrones, cuellos de botella y oportunidades de optimización en el proceso comercial.